In [1]:
import numpy as np
import pandas as pd
import os
from astropy.io import fits
import ccdproc as cp

In [2]:
main_dir = "/Volumes/LaCie/School/ASTR_301/A2/"
data_dir = main_dir+"data/"
out_dir = main_dir+"out/"

## To-Do

1. Median Bias Frame Creator

1. Compute the dark current:
    
    1. stats on overscan region after bias correction
    
1. Apply overscan corr.

1. Dome Flats

## Master Table

In [3]:
df = pd.read_csv(data_dir+'Tek2007Nov05.log', sep=",", 
                header=None, names= ["Filename", "AIRMASS","EXPTIME", "Filter", "Type"])

## Creating a Master Bias

In [4]:
bias_list = (df["Filename"][df["EXPTIME"] == 0])

In [ ]:
bias_frames = []

for image in bias_list:
    print("Inspecting Image ... %s")%(image)
    
    try:
        frame = cp.fits_ccddata_reader(data_dir+image+".gz", hdu=0, unit="adu")
        hdulist = fits.open(data_dir+image+".gz") 
    
    except IOError:
        frame = cp.fits_ccddata_reader(data_dir+image, hdu=0, unit="adu")
        hdulist = fits.open(data_dir+image) 
   
    #header = hdulist[0].header
    data = hdulist[0].data
        
    #frame = cp.subtract_overscan(frame, 
    #                            fits_section=frame.header["BIASSEC"],
    #                            overscan_axis=(0,1),
    #                           median=True)
    
    os_region = data[2049:2080, 1:2068]
    
    print("\t Median : %s")%(np.median(os_region))
    
    bias_frames.append(frame)

In [ ]:
master_bias = cp.combine(bias_frames,method="median")
cp.fits_ccddata_writer(frame, out_dir+"master_bias_2.fits")

## Inspecting Bias

In [17]:
bias = fits.open(out_dir+"master_bias_2.fits")
bias_data = bias[0].data


k = 0
for row in range(len(bias_data)):
    for column in range(len(bias_data)):
        if bias_data[row][column] >= 250:
            k += 1

tot_pix = len(bias_data)**2
perc =  (float(k) / float(tot_pix)) * 100.0
print("Number of bad pixels: %s")%k
print("Total numer of pixels: %s")%tot_pix
print("Percentage: %s")%perc

Number of bad pixels: 80133
Total numer of pixels: 4276624
Percentage: 1.87374433665


## Dark Current